In [1]:
import numpy as np
import pandas as pd
import biogeme
import sklearn

In [2]:
#Benchmark

%run -i "binary_specific_netherlands.py"

           Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_CAR   -0.798     0.27   -2.95  0.00315         0.275         -2.9   
BETA_COST -0.113   0.0232   -4.85 1.21e-06        0.0241        -4.67   
BETA_TT    -1.33    0.344   -3.86 0.000115         0.354        -3.75   

           Rob. p-value  
ASC_CAR         0.00379  
BETA_COST      3.03e-06  
BETA_TT         0.00018  
Nbr of observations: 228
LL(0) =    -123.133
LL(beta) = -123.133
rho bar square = -0.0244
Output file: binary_specific_netherlands~03.html


In [3]:
ned_data_rp = pd.read_table("Data/netherlandsRP.dat")

X = ned_data_rp[['car_cost', 'rail_cost', 'car_ivtt', 'car_walk_time', 'rail_ivtt', 'rail_acc_time', 'rail_egr_time']]
X_updt = X.copy()
X_updt['car_tt'] = X['car_ivtt'] +X['car_walk_time']
X_updt['rail_tt'] = X['rail_ivtt'] +X['rail_acc_time'] +X['rail_egr_time']

rate_G2E = 0.44378022
X_updt['car_cost'] = X_updt['car_cost'] * rate_G2E
X_updt['rail_cost'] = X_updt['rail_cost'] * rate_G2E
X_updt = X_updt.drop(columns=['car_ivtt', 'car_walk_time', 'rail_ivtt', 'rail_acc_time', 'rail_egr_time'])

y = ned_data_rp['choice']

In [4]:
X_updt

,car_cost,rail_cost,car_tt,rail_tt
0,2.22,17.8,1.17,2.5
1,3.99,15.5,1.52,2.02
2,5.1,10.7,1.97,2.02
3,3.7,3.46,2,1.75
4,2.22,12.4,1.27,2.03
...,...,...,...,...
223,6.21,16.9,1.4,2.42
224,7.77,17.8,1.33,2.33
225,2.77,12,1.83,2.08
226,9.62,19.5,1.27,1.75


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(X_updt, y, test_size=0.20, random_state=42)

In [6]:
import xgboost as xgb 

xgb_constrained = xgb.XGBClassifier(#objective='binary:logistic', 
                                    #learning_rate = 0.2,
                                    n_estimators = 5, 
                                    max_depth = 5, 
                                    random_state = 42,
                                    monotone_constraints = (-1, -1, -1, -1),
                                    interaction_constraints = [["car_cost"], ["rail_cost"], ["car_tt"], ["rail_tt"]]
                                   )

In [7]:
#xgb_constrained.get_params()

In [8]:
eval_set = [(X_validate, y_validate)]
xgb_constrained.fit(X_train, y_train, eval_set=eval_set)

[0]	validation_0-logloss:0.69091
[1]	validation_0-logloss:0.67756
[2]	validation_0-logloss:0.67163
[3]	validation_0-logloss:0.67809
[4]	validation_0-logloss:0.68440


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=[['car_cost'], ['rail_cost'], ['car_tt'],
                                       ['rail_tt']],
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=(-1, -1, -1, -1), n_estimators=5,
              n_jobs=None, num_parallel_tree=None, predictor=None,
              random_state=42, ...)

In [9]:
y_pred=xgb_constrained.predict(X_validate)

from sklearn.metrics import accuracy_score

accuracy_score(y_validate, y_pred)

0.5652173913043478

In [11]:
from xgbfir_modified import saveXgbFI

saveXgbFI(xgb_constrained)

In [134]:
import xgbfir

xgbfir.saveXgbFI(xgb_constrained)